# Visual Dynamics

This code is the implementation of `Visual Dynamics: Probabilistic Future Frame Synthesis via Cross Convolutional Networks`

In [2]:
import numpy as np
import tensorflow as tf

In [47]:
img1 = tf.placeholder(shape=(None,256,256,3), dtype=tf.float32)
img2 = tf.placeholder(shape=(None,256,256,3), dtype=tf.float32)
img1_128 = tf.nn.max_pool(img1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
img2_128 = tf.nn.max_pool(img2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
img1_64 = tf.nn.max_pool(img1, ksize=[1,4,4,1], strides=[1,4,4,1], padding='VALID')
img1_32 = tf.nn.max_pool(img1, ksize=[1,8,8,1], strides=[1,8,8,1], padding='VALID')

#Motion encoder
x1 = tf.concat([img1_128,img2_128], axis=3)

#First convolution: 5x5x96
weights = tf.Variable(tf.random_normal([5,5,6,96]))
bias = tf.Variable(tf.zeros([96,]))
logits = tf.nn.conv2d(x1, filter=weights, strides=[1,1,1,1], padding='SAME')
logits = tf.add(logits, bias)
logits = tf.nn.relu(logits)

#Second convolution: 5x5x96
weights = tf.Variable(tf.random_normal([5,5,96,96]))
bias = tf.Variable(tf.zeros([96,]))
logits = tf.nn.conv2d(logits, filter=weights, strides=[1,1,1,1], padding='SAME')
logits = tf.add(logits, bias)
logits = tf.nn.max_pool(logits, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits = tf.nn.relu(logits)

#Third convolution: 5x5x128
weights = tf.Variable(tf.random_normal([5,5,96,128]))
bias = tf.Variable(tf.zeros([128,]))
logits = tf.nn.conv2d(logits, filter=weights, strides=[1,1,1,1], padding='SAME')
logits = tf.add(logits, bias)
logits = tf.nn.max_pool(logits, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits = tf.nn.relu(logits)

#Forth convolution: 5x5x128
weights = tf.Variable(tf.random_normal([5,5,128,128]))
bias = tf.Variable(tf.zeros([128,]))
logits = tf.nn.conv2d(logits, filter=weights, strides=[1,1,1,1], padding='VALID')
logits = tf.add(logits, bias)
logits = tf.nn.max_pool(logits, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits = tf.nn.relu(logits)

#Fifth convolution: 5x5x256
weights = tf.Variable(tf.random_normal([5,5,128,256]))
bias = tf.Variable(tf.zeros([256,]))
logits = tf.nn.conv2d(logits, filter=weights, strides=[1,1,1,1], padding='VALID')
logits = tf.add(logits, bias)
logits = tf.nn.max_pool(logits, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits = tf.nn.relu(logits)

#Sixth convolution: 5x5x256
weights = tf.Variable(tf.random_normal([5,5,256,256]))
bias = tf.Variable(tf.zeros([256,]))
logits = tf.nn.conv2d(logits, filter=weights, strides=[1,1,1,1], padding='SAME')
logits = tf.add(logits, bias)
logits = tf.nn.relu(logits)

logits = tf.contrib.layers.flatten(logits)

mean, variance = tf.split(logits, 2, axis=1)

kernel = tf.reshape(variance, shape=[tf.shape(logits)[0],5,5,128])

kernel_w = tf.Variable(tf.random_normal([5,5,128,128]))
kernel_bias = tf.Variable(tf.random_normal([128]))
kernel = tf.nn.conv2d(kernel, filter=kernel_w, strides=[1,1,1,1], padding='SAME')
kernel = tf.add(kernel, kernel_bias)
kernel = tf.nn.relu(kernel)

kernel_w = tf.Variable(tf.random_normal([5,5,128,128]))
kernel_bias = tf.Variable(tf.random_normal([128]))
kernel = tf.nn.conv2d(kernel, filter=kernel_w, strides=[1,1,1,1], padding='SAME')
kernel = tf.add(kernel, kernel_bias)
kernel = tf.nn.relu(kernel)

set1, set2, set3, set4 = tf.split(kernel, 4, axis=3)

#Img encoder
# for img 256x256
logits_img256_encoder_w = tf.Variable(tf.random_normal([5,5,3,64])) 
logits_img256_encoder_b = tf.Variable(tf.zeros([64]))
logits_img256_encoder = tf.nn.conv2d(img1, filter=logits_img256_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img256_encoder = tf.add(logits_img256_encoder, logits_img256_encoder_b)
logits_img256_encoder = tf.nn.relu(logits_img256_encoder)

logits_img256_encoder_w = tf.Variable(tf.random_normal([5,5,64,64])) 
logits_img256_encoder_b = tf.Variable(tf.zeros([64]))
logits_img256_encoder = tf.nn.conv2d(logits_img256_encoder, filter=logits_img256_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img256_encoder = tf.add(logits_img256_encoder, logits_img256_encoder_b)
logits_img256_encoder = tf.nn.max_pool(logits_img256_encoder, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits_img256_encoder = tf.nn.relu(logits_img256_encoder)

logits_img256_encoder_w = tf.Variable(tf.random_normal([5,5,64,64])) 
logits_img256_encoder_b = tf.Variable(tf.zeros([64]))
logits_img256_encoder = tf.nn.conv2d(logits_img256_encoder, filter=logits_img256_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img256_encoder = tf.add(logits_img256_encoder, logits_img256_encoder_b)
logits_img256_encoder = tf.nn.max_pool(logits_img256_encoder, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits_img256_encoder = tf.nn.relu(logits_img256_encoder)

logits_img256_encoder_w = tf.Variable(tf.random_normal([5,5,64,32])) 
logits_img256_encoder_b = tf.Variable(tf.zeros([32]))
logits_img256_encoder = tf.nn.conv2d(logits_img256_encoder, filter=logits_img256_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img256_encoder = tf.add(logits_img256_encoder, logits_img256_encoder_b)
logits_img256_encoder = tf.nn.relu(logits_img256_encoder)

# for img 128x128
logits_img128_encoder_w = tf.Variable(tf.random_normal([5,5,3,64])) 
logits_img128_encoder_b = tf.Variable(tf.zeros([64]))
logits_img128_encoder = tf.nn.conv2d(img1_128, filter=logits_img128_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img128_encoder = tf.add(logits_img128_encoder, logits_img128_encoder_b)
logits_img128_encoder = tf.nn.relu(logits_img128_encoder)

logits_img128_encoder_w = tf.Variable(tf.random_normal([5,5,64,64])) 
logits_img128_encoder_b = tf.Variable(tf.zeros([64]))
logits_img128_encoder = tf.nn.conv2d(logits_img128_encoder, filter=logits_img128_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img128_encoder = tf.add(logits_img128_encoder, logits_img128_encoder_b)
logits_img128_encoder = tf.nn.max_pool(logits_img128_encoder, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits_img128_encoder = tf.nn.relu(logits_img128_encoder)

logits_img128_encoder_w = tf.Variable(tf.random_normal([5,5,64,64])) 
logits_img128_encoder_b = tf.Variable(tf.zeros([64]))
logits_img128_encoder = tf.nn.conv2d(logits_img128_encoder, filter=logits_img128_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img128_encoder = tf.add(logits_img128_encoder, logits_img128_encoder_b)
logits_img128_encoder = tf.nn.max_pool(logits_img128_encoder, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits_img128_encoder = tf.nn.relu(logits_img128_encoder)

logits_img128_encoder_w = tf.Variable(tf.random_normal([5,5,64,32])) 
logits_img128_encoder_b = tf.Variable(tf.zeros([32]))
logits_img128_encoder = tf.nn.conv2d(logits_img128_encoder, filter=logits_img128_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img128_encoder = tf.add(logits_img128_encoder, logits_img128_encoder_b)
logits_img128_encoder = tf.nn.relu(logits_img128_encoder)

# for img 64x64
logits_img64_encoder_w = tf.Variable(tf.random_normal([5,5,3,64])) 
logits_img64_encoder_b = tf.Variable(tf.zeros([64]))
logits_img64_encoder = tf.nn.conv2d(img1_64, filter=logits_img64_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img64_encoder = tf.add(logits_img64_encoder, logits_img64_encoder_b)
logits_img64_encoder = tf.nn.relu(logits_img64_encoder)

logits_img64_encoder_w = tf.Variable(tf.random_normal([5,5,64,64])) 
logits_img64_encoder_b = tf.Variable(tf.zeros([64]))
logits_img64_encoder = tf.nn.conv2d(logits_img64_encoder, filter=logits_img64_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img64_encoder = tf.add(logits_img64_encoder, logits_img64_encoder_b)
logits_img64_encoder = tf.nn.max_pool(logits_img64_encoder, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits_img64_encoder = tf.nn.relu(logits_img64_encoder)

logits_img64_encoder_w = tf.Variable(tf.random_normal([5,5,64,64])) 
logits_img64_encoder_b = tf.Variable(tf.zeros([64]))
logits_img64_encoder = tf.nn.conv2d(logits_img64_encoder, filter=logits_img64_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img64_encoder = tf.add(logits_img64_encoder, logits_img64_encoder_b)
logits_img64_encoder = tf.nn.max_pool(logits_img64_encoder, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits_img64_encoder = tf.nn.relu(logits_img64_encoder)

logits_img64_encoder_w = tf.Variable(tf.random_normal([5,5,64,32])) 
logits_img64_encoder_b = tf.Variable(tf.zeros([32]))
logits_img64_encoder = tf.nn.conv2d(logits_img64_encoder, filter=logits_img64_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img64_encoder = tf.add(logits_img64_encoder, logits_img64_encoder_b)
logits_img64_encoder = tf.nn.relu(logits_img64_encoder)

# for 32x32
logits_img32_encoder_w = tf.Variable(tf.random_normal([5,5,3,64])) 
logits_img32_encoder_b = tf.Variable(tf.zeros([64]))
logits_img32_encoder = tf.nn.conv2d(img1_32, filter=logits_img32_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img32_encoder = tf.add(logits_img32_encoder, logits_img32_encoder_b)
logits_img32_encoder = tf.nn.relu(logits_img32_encoder)

logits_img32_encoder_w = tf.Variable(tf.random_normal([5,5,64,64])) 
logits_img32_encoder_b = tf.Variable(tf.zeros([64]))
logits_img32_encoder = tf.nn.conv2d(logits_img32_encoder, filter=logits_img32_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img32_encoder = tf.add(logits_img32_encoder, logits_img32_encoder_b)
logits_img32_encoder = tf.nn.max_pool(logits_img32_encoder, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits_img32_encoder = tf.nn.relu(logits_img32_encoder)

logits_img32_encoder_w = tf.Variable(tf.random_normal([5,5,64,64])) 
logits_img32_encoder_b = tf.Variable(tf.zeros([64]))
logits_img32_encoder = tf.nn.conv2d(logits_img32_encoder, filter=logits_img32_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img32_encoder = tf.add(logits_img32_encoder, logits_img32_encoder_b)
logits_img32_encoder = tf.nn.max_pool(logits_img32_encoder, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
logits_img32_encoder = tf.nn.relu(logits_img32_encoder)

logits_img32_encoder_w = tf.Variable(tf.random_normal([5,5,64,32])) 
logits_img32_encoder_b = tf.Variable(tf.zeros([32]))
logits_img32_encoder = tf.nn.conv2d(logits_img32_encoder, filter=logits_img32_encoder_w, strides=[1,1,1,1], padding='SAME')
logits_img32_encoder = tf.add(logits_img32_encoder, logits_img32_encoder_b)
logits_img32_encoder = tf.nn.relu(logits_img32_encoder)

#cross_conv_64 = tf.nn.conv2d(logits_img256_encoder, filter=set1, strides=[1,1,1,1], padding='SAME')
#cross_conv_32 = tf.multiply(logits_img128_encoder, set2) 
#cross_conv_16 = tf.multiply(logits_img64_encoder, set3) 
#cross_conv_8 = tf.multiply(logits_img32_encoder, set4) 

ValueError: Dimensions must be equal, but are 32 and 5 for 'Conv2D_466' (op: 'Conv2D') with input shapes: [?,64,64,32], [?,5,5,32].